In [ ]:
import polars as pl
import os, json
from dotenv import load_dotenv
import gcsfs
import datetime
from google.cloud import storage

load_dotenv()

True

In [23]:
# Create a proper service-account JSON from .env and set it for the running kernel
creds = {
    "type": "service_account",
    "project_id": os.getenv("GCP_PROJECT"),
    "private_key_id": os.getenv("GCP_PRIVATE_KEY_ID"),
    "private_key": os.getenv("GCP_PRIVATE_KEY", "").replace("\\n", "\n"),
    "client_email": os.getenv("GCP_EMAIL"),
    "client_id": os.getenv("GCP_CLIENT_ID", ""),
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "",
}

creds_path = os.path.abspath("gcp-creds.json")
with open(creds_path, "w", encoding="utf-8") as f:
    json.dump(creds, f)
print("Wrote", creds_path, "(DO NOT COMMIT)")

# Make this kernel use the creds immediately
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

Wrote c:\Users\emili\OneDrive\Desktop\Generative AI\gen-ai\src\research\jacob\gcp-creds.json (DO NOT COMMIT)


In [25]:
client = storage.Client.from_service_account_json(creds_path)
bucket = client.bucket("gen-ai-tu")

for b in bucket.list_blobs(prefix="news/raw/year=2025/month=11/day=01/main_lang=en/"):
    print(b.name)

news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101004549-04954.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101021758-04955.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101033728-04956.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101050844-04957.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101062720-04958.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101074035-04959.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101085453-04960.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101100459-04961.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS-20251101110747-04962.warc.gz/0.parquet
news/raw/year=2025/month=11/day=01/main_lang=en/filename=CC-NEWS

In [26]:
def download_parquet(full_path, dest="parquet_files"):
    import os

    os.makedirs(dest, exist_ok=True)

    # Keep the original filename to avoid misnaming
    filename = full_path.split("/")[-1]  # '0.parquet'
    local_path = os.path.join(dest, filename)

    blob = bucket.blob(full_path)
    blob.download_to_filename(local_path)

    print("Downloaded:", local_path)
    return local_path


def download_all_parquet_for_date(year, month, day, dest="parquet_files"):
    # Prefix only matches the exact date and main_lang=en
    prefix = f"news/raw/year={year}/month={month:02d}/day={day:02d}/main_lang=en/"
    print("Searching prefix:", prefix)

    blobs = bucket.list_blobs(prefix=prefix)

    downloaded = []

    for blob in blobs:
        # Only download the actual parquet file (usually named '0.parquet')
        if blob.name.endswith("0.parquet"):
            downloaded.append(download_parquet(blob.name, dest=dest))

    print(f"\nDownloaded {len(downloaded)} parquet files.")
    return downloaded


# Example usage:
downloaded_files = download_all_parquet_for_date(2025, 11, 4)

Searching prefix: news/raw/year=2025/month=11/day=04/main_lang=en/
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parquet
Downloaded: parquet_files\0.parqu